# At Home Solar Charging notebooks: Exploring how EV charging aligns with rooftop solar generation by homes

<p>We will be using Pecan Street Inc. data from Dataport to determine how electric vehicle charging aligns with rooftop solar generation.
    
<br>Data from 24 homes with fairly complete data for the year 2018 is used to explore this question.
    
<br>
Pecans Streets data can be obtained by applying for a dataport account at https://www.dataport.pecanstreet.org.</p>


In [ ]:
#import packages
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sqlalchemy as sqla
import os
from config.read_config import get_database_config
import numpy as np
import sys
%matplotlib inline
sys.executable  # shows you your path to the python you're using

In [ ]:
# read in db credentials from config/config.txt
# * make sure you add those to the config/config.txt file! *

database_config = get_database_config("./config/config.txt")

In [ ]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format(database_config['username'],
                                                                     database_config['password'],
                                                                     database_config['hostname'],
                                                                     database_config['port'],
                                                                     database_config['database']
                                                                     ))

In [ ]:
#Select a list of Texas homes from dataport metadata having CAR and solar configured and also has data for year 2018.
query = """select distinct dataid from other_datasets.metadata 
                                          where car1='yes' and solar='yes' 
                                          and egauge_1min_min_time < '2018-01-01' 
                                          and egauge_1min_max_time > '2019-01-01'
                                          and state='Texas'
                                          and (egauge_1min_data_availability like '100%' 
                                               or 
                                               egauge_1min_data_availability like '99%')
                                          ;
         """

df = pd.read_sql_query(sqla.text(query), engine)

In [ ]:
# grab dataids and convert them to a string to put into the SQL query
dataids_list = df['dataid'].tolist()
print("{} dataids selected listed here:".format(len(dataids_list)))
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str
dataids_list

In [ ]:
#Check data completeness for dataids selected from metadata above.

query2 = """select dataid,count(*) total_rec from electricity.eg_realpower_1min 
            where dataid in ({})""".format(dataids_str)
query2 = query2 + """ and localminute >= '2018-01-01' and localminute < '2019-01-01' group by 1"""

df2 = pd.read_sql_query(sqla.text(query2), engine)

In [ ]:
#Select homes with atleast 90% data availability for year 2018.
df2['perc'] = (df2['total_rec']/525600)*100
final_dataids = df2[df2['perc'] >= 90]
final_dataids['dataid'].count()

In [ ]:
# Pull data for homes
final_dataids_list = final_dataids['dataid'].tolist()
print("{} dataids selected listed here:".format(len(final_dataids_list)))
final_dataids_str = ','.join(list(map(str, final_dataids_list)))
final_dataids_str
final_dataids_list

In [ ]:
#fall
fall = """select localminute::timestamp,car1,solar,grid 
               from electricity.eg_realpower_1min 
               where localminute >= '2018-09-01' and localminute <  '2018-12-01' """
fall = fall + """AND dataid in ({})""".format(final_dataids_str)

fall_df = pd.read_sql_query(sqla.text(fall), engine)

fall_df.head(15)

In [ ]:
#spring
spring = """select localminute::timestamp,car1,solar,grid 
               from electricity.eg_realpower_1min 
               where localminute >= '2018-03-01' and localminute <  '2018-06-01' """
spring = spring + """AND dataid in ({})""".format(final_dataids_str)

spring_df = pd.read_sql_query(sqla.text(spring), engine)

In [ ]:
#summer
summer = """select localminute::timestamp,car1,solar,grid 
               from electricity.eg_realpower_1min 
               where localminute >= '2018-06-01' and localminute <  '2018-09-01' """
summer = summer + """AND dataid in ({})""".format(final_dataids_str)

# create a dataframe with the data from the sql query
summer_df = pd.read_sql_query(sqla.text(summer), engine)

In [ ]:
#winter
winter = """select localminute::timestamp,car1,solar,grid 
               from electricity.eg_realpower_1min 
               where localminute >= '2018-12-01' and localminute <  '2019-03-01' """
winter = winter + """AND dataid in ({})""".format(final_dataids_str)

# create a dataframe with the data from the sql query
winter_df = pd.read_sql_query(sqla.text(winter), engine)

In [ ]:
# data analysis for summer

#replace null with 0's
summer_df = summer_df.fillna(0)
# convert localminute to pandas daytime type
summer_df['datetime'] = pd.to_datetime(summer_df['localminute'])

summer_df = summer_df.set_index('datetime')

#create hour column
summer_df['hr'] = summer_df.index.hour

#create new dataframes with solar, car1, and hour column
summer_df_new = pd.DataFrame(summer_df, columns = ['solar', 'car1', 'hr'])

# group data based on hour and get avg
summer_df_grouped = summer_df_new.groupby(['hr']).mean()
plot_summer = summer_df_grouped.reset_index()

In [ ]:
# plot summer
plt.figure(figsize=(20,10))
plt.plot(plot_summer['hr'], plot_summer['car1'], label="car1")
plt.plot(plot_summer['hr'], plot_summer['solar'], label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
# Total car1 usuage powered by solar in summer
total_car1_summer = summer_df_new['car1'].sum()
solar_car1_summer = summer_df_new.loc[summer_df_new['car1'] < summer_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_summer = (solar_car1_summer/total_car1_summer) * 100
car1_powered_by_solar_summer

In [ ]:
# data analysis for fall

#replace null with 0's
fall_df = fall_df.fillna(0)
# convert localminute to pandas daytime type
fall_df['datetime'] = pd.to_datetime(fall_df['localminute'])

fall_df = fall_df.set_index('datetime')

#create hour column
fall_df['hr'] = fall_df.index.hour

#create new dataframes with solar, car1, and hour column
fall_df_new = pd.DataFrame(fall_df, columns = ['solar', 'car1', 'hr'])

# group data based on hour and get avg
fall_df_grouped = fall_df_new.groupby(['hr']).mean()
plot_fall = fall_df_grouped.reset_index()

In [ ]:
# plot fall
plt.figure(figsize=(20,10))
plt.plot(plot_fall['hr'], plot_fall['car1'], label="car1")
plt.plot(plot_fall['hr'], plot_fall['solar'], label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
# Total car1 usuage powered by solar in fall
total_car1_fall = fall_df_new['car1'].sum()
solar_car1_fall = fall_df_new.loc[fall_df_new['car1'] < fall_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_fall = (solar_car1_fall/total_car1_fall) * 100
car1_powered_by_solar_fall

In [ ]:
#data analysis for spring

#replace null with 0's 
spring_df = spring_df.fillna(0)


# convert localminute to pandas datetime type
spring_df['datetime'] = pd.to_datetime(spring_df['localminute'])

spring_df = spring_df.set_index('datetime')

#create hour column. We will be calculating average hourly load for spring.
spring_df['hr'] = spring_df.index.hour

#create new dataframes with only solar, car1 and hour column
spring_df_new = pd.DataFrame(spring_df, columns = ['solar', 'car1','hr'])

#group data based on hour and take avg
spring_df_grouped = spring_df_new.groupby(['hr']).mean()
plot_spring = spring_df_grouped.reset_index()

In [ ]:
#plot spring
plt.figure(figsize=(20,10))
plt.plot(plot_spring['hr'],plot_spring['car1'],label="car1")
plt.plot(plot_spring['hr'],plot_spring['solar'],label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
#Total car1 usage powered by PV system in spring
total_car1_spring = spring_df_new['car1'].sum()
solar_car1_spring = spring_df_new.loc[spring_df_new['car1'] < spring_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_spring = (solar_car1_spring/total_car1_spring)*100
car1_powered_by_solar_spring

In [ ]:
#data analysis for winter

#replace null with 0's 
winter_df = winter_df.fillna(0)

# convert localminute to pandas datetime type
winter_df['datetime'] = pd.to_datetime(winter_df['localminute'])

winter_df = winter_df.set_index('datetime')

#create hour column. We will be calculating average hourly load for winter.
winter_df['hr'] = winter_df.index.hour

#create new dataframes with only solar, car1 and hour column
winter_df_new = pd.DataFrame(winter_df, columns = ['solar', 'car1','hr'])

#group data based on hour and take avg
winter_df_grouped = winter_df_new.groupby(['hr']).mean()
plot_winter = winter_df_grouped.reset_index()

In [ ]:
#plot winter
plt.figure(figsize=(20,10))
plt.plot(plot_winter['hr'],plot_winter['car1'],label="car1")
plt.plot(plot_winter['hr'],plot_winter['solar'],label="solar")
plt.xticks(np.arange(0, 24, 1.0))
plt.xlabel('hour')
plt.legend()

plt.show()

In [ ]:
#Total car1 usage powered by PV system in winter
total_car1_winter = winter_df_new['car1'].sum()
solar_car1_winter = winter_df_new.loc[winter_df_new['car1'] < winter_df_new['solar'], 'car1'].sum()

car1_powered_by_solar_winter = (solar_car1_winter/total_car1_winter) * 100
car1_powered_by_solar_winter

In [ ]:
import statistics
statistics.mean([car1_powered_by_solar_winter, car1_powered_by_solar_spring, car1_powered_by_solar_summer, car1_powered_by_solar_fall])

## Observations:
From the above graphs through out the year most homes started EV charging arount 9PM to 9:30PM and ended charging around 6AM (outside of solar peak hours). The average percentage of solar powered used for EV charging is around 4.4 %